In [9]:
#| default_exp trait

In [10]:
#|export
from chewc.config import Population, StaticConfig, DynamicConfig, Trait
import jax
import jax.numpy as jnp
from typing import Optional
from functools import partial


In [11]:
#| export

def add_trait(
    key: jax.random.PRNGKey,
    founder_pop: Population,
    s_config: StaticConfig,
    d_config: DynamicConfig,
    n_qtl_per_chr: int,
    mean: jnp.ndarray,      # (n_traits,)
    var: jnp.ndarray,       # (n_traits,)
    Sigma: Optional[jnp.ndarray] = None,  # (n_traits, n_traits) covariance for correlated traits
) -> DynamicConfig:
    """Create one trait (or multiple traits if mean/var have >1 dims) and scale to target additive var.
    Returns a new DynamicConfig with this trait appended (here we replace for simplicity)."""
    key, qtl_key, effect_key = jax.random.split(key, 3)
    n_total_qtl = n_qtl_per_chr * s_config.n_chr

    # choose QTL loci across the whole genome
    all_loci_indices = jnp.arange(s_config.n_chr * s_config.n_loci_per_chr)
    qtl_loc_flat = jax.random.choice(qtl_key, all_loci_indices, (n_total_qtl,), replace=False)
    qtl_chromosome, qtl_position = jnp.divmod(jnp.sort(qtl_loc_flat), s_config.n_loci_per_chr)

    n_traits = int(mean.shape[0])
    raw_effects = jax.random.normal(effect_key, (n_total_qtl, n_traits))
    if Sigma is not None:
        L = jnp.linalg.cholesky(Sigma)
        effects = raw_effects @ L
    else:
        effects = raw_effects

    # compute TBV in founders to scale to target var
    founder_dosage = compute_dosage(founder_pop)  # (N, C, L)
    G_Q = founder_dosage[:, qtl_chromosome, qtl_position]                    # (N, Q)
    gvs = G_Q @ effects                                                      # (N, n_traits)

    # scale per-trait to match target additive variance in founders
    scale = jnp.sqrt(var / (jnp.var(gvs, axis=0) + 1e-8))                    # (n_traits,)
    intercept = mean - jnp.mean(gvs, axis=0) * scale                         # (n_traits,)
    final_eff = effects * scale                                              # (Q, n_traits)

    trait = Trait(qtl_chromosome, qtl_position, final_eff, intercept)
    return d_config.replace(traits=(trait,))

from typing import Union, Optional


def compute_dosage(pop: Population) -> jnp.ndarray:
    # (N, C, P, L) -> sum over ploidy -> (N, C, L) float32
    return jnp.sum(pop.geno, axis=2).astype(jnp.float32)


def gather_qtl_dosage(
    dosage: jnp.ndarray,
    trait: Trait,
    *,
    n_loci_per_chr: int,
) -> jnp.ndarray:
    # dosage: (N, C, L) -> (N, Q) at QTL positions
    N, C, L = dosage.shape
    flat = dosage.reshape(N, C * L)
    lin = trait.qtl_chromosome * n_loci_per_chr + trait.qtl_position
    return jnp.take(flat, lin, axis=1)




@partial(jax.jit, static_argnames=("n_loci_per_chr", "trait_index", "broad_sense"))
def set_pheno_h2(
    key: jax.random.PRNGKey,
    pop: Population,
    d_config: DynamicConfig,
    h2: Union[float, jnp.ndarray],
    *,
    n_loci_per_chr: int,
    trait_index: int = 0,
    broad_sense: bool = False,
    active_mask: Optional[jnp.ndarray] = None,  # (N,), optional
):
    """
    Compute phenotypes at a target heritability h2.

    Returns:
        pheno: (N, n_traits)
        bv:    (N, n_traits)  # additive breeding values (TBV)
        gv:    (N, n_traits)  # genotypic values; == bv for additive-only model
        genetic_variance: (n_traits,)
        var_e: (n_traits,)
    JIT-friendly (no boolean indexing with unknown-size outputs).
    """
    # Dosage: (N, C, L)
    dosage = compute_dosage(pop)  # your existing helper

    # Select which Trait object to use (you currently use the first one)
    trait = d_config.traits[trait_index]

    # Gather QTL dosage: (N, Q)
    G_Q = gather_qtl_dosage(dosage, trait, n_loci_per_chr=n_loci_per_chr)

    # BV/TBV: (N, n_traits)
    # (Q, n_traits) @ (N, Q) -> (N, n_traits) plus intercept broadcast
    bv = G_Q @ trait.qtl_effects + trait.intercept  # additive-only TBV

    # For now (additive model), GV == BV; extend here for dominance/epistasis later
    gv = bv

    # Mask handling (optional)
    N = pop.geno.shape[0]
    if active_mask is None:
        active_mask = jnp.ones((N,), dtype=bool)
    mask_2d = active_mask[:, None]  # broadcast across traits

    # Target genetic variance source
    genetic_component = gv if broad_sense else bv

    # Variance per trait over active individuals; JIT-safe via where=
    genetic_variance = jnp.var(genetic_component, axis=0, where=mask_2d)

    # Noise variance from h2 (scalar or per-trait)
    h2_arr = jnp.asarray(h2, dtype=genetic_variance.dtype)
    var_e = genetic_variance * (1.0 / (h2_arr + 1e-8) - 1.0)
    var_e = jnp.maximum(var_e, 0.0)

    # Add environmental noise; broadcasting var_e over individuals
    env = jax.random.normal(key, gv.shape) * jnp.sqrt(var_e)

    pheno = gv + env

    # If a mask was provided, write NaNs for inactive individuals (keeps shapes static)
    nan = jnp.array(jnp.nan, dtype=pheno.dtype)
    bv    = jnp.where(mask_2d, bv,    nan)
    gv    = jnp.where(mask_2d, gv,    nan)
    pheno = jnp.where(mask_2d, pheno, nan)

    return pheno, bv, gv, genetic_variance, var_e




In [12]:
#| hide
import nbdev; nbdev.nbdev_export()